Uses `v2` dataset.

In [ ]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

import datetime
import sys
from os import path, environ
import pickle
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from tqdm.auto import tqdm, trange
from bdx import get_trend

from utils import contiguous_sequences
from plotting import model_surface, plot_surface
from systems import Condenser
from baseline_control import SimpleFeedbackController

chiller_file_1 = path.join(environ['DATADIR'],
                         'EngineeringScienceBuilding',
                         '2422_ESB_HVAC_1.csv')
chiller_file_2 = path.join(environ['DATADIR'],
                         'EngineeringScienceBuilding',
                         '2841_ESB_HVAC_2.csv')

plot_path = path.join('..', 'docs', 'img')
bin_path = './bin/'

In [ ]:
# Choosing which chiller to use
chiller_file = chiller_file_2
# Data selection 'all' or 'chiller_on' or 'fan_on'
MODE = 'chiller_on'
# Read pre-processed data:
# Pytorch uses float32 as default type for weights etc,
# so input data points are also read in the same type.
df = pd.read_csv(chiller_file, index_col='time',
                 parse_dates=['time'], dtype=np.float32)
print('Original length: {} Records'.format(len(df)))
# # These fields were not populated until 2020-07-01, so leaving then out of analysis
# df.drop(['PowFanA', 'PowFanB', 'FlowCond', 'PowChiP', 'PerFreqConP', 'PowConP'], axis='columns', inplace=True)
df.drop(['FlowCond', 'PowChiP', 'PerFreqConP'], axis='columns', inplace=True)

df.dropna(inplace=True)
if MODE == 'chiller_on':
    df = df[df['RunChi'] != 0]
if MODE == 'fan_on':
    df = df[(df['RunFanA'] != 0.) | df['RunFanB'] != 0.]
print('Processed length: {} Records'.format(len(df)))

In [ ]:
df.describe()

## Condenser model

State variables (12):

`'TempCondIn', 'TempCondOut', 'TempEvapOut', 'PowChi', 'PowFanA', 'PowFanB', 'PowConP', 'TempEvapIn', 'TempAmbient', 'TempWetBulb', 'PressDiffEvap', 'PressDiffCond'`

Action variables (1):

`'TempCondInSetpoint'`

Output variables (3):

`'TempCondIn', 'TempCondOut', 'TempEvapOut', 'PowChi', 'PowFanA', 'PowFanB', 'PowConP'`

Model:

`[Action, State] --> [Output]`

In [ ]:
actionvars = ['TempCondInSetpoint']
statevars = ['TempCondIn', 'TempCondOut', 'TempEvapOut',
             'PowChi', 'PowFanA', 'PowFanB', 'PowConP',
             'TempEvapIn', 'TempAmbient', 'TempWetBulb', 'PressDiffEvap', 'PressDiffCond']
inputs =  actionvars + statevars 
outputs = ['TempCondIn', 'TempCondOut', 'TempEvapOut',
           'PowChi', 'PowFanA', 'PowFanB', 'PowConP']
lag = (1, 1, 1, 1, 1, 1, 1)    # 0, 1, 2, 3, 4, ...

df_in = pd.DataFrame(columns=inputs, index=df.index)
df_in['TempCondInSetpoint'] = np.clip(df['TempWetBulb'] - 4, a_min=65, a_max=None)  # approach controller
df_in[inputs[1:]] = df[inputs[1:]]

df_out = pd.DataFrame(columns=outputs, index=df.index)
df_out[outputs] = df[outputs]

idx_list = contiguous_sequences(df.index, pd.Timedelta(5, unit='min'), filter_min=10)

# Create dataframes of contiguous sequences with a delay
# of 1 time unit to indicate causality input -> outputs
dfs_in, dfs_out = [], []
for idx in idx_list:
    dfs_in.append(df_in.loc[idx[:-max(lag) if max(lag) > 0 else None]])
    cols = []
    for l, c in zip(lag, outputs):
        window = slice(l, None if l==max(lag) else -(max(lag)-l))
        series = df_out[c].loc[idx[window]]
        cols.append(series.values)
        if l == min(lag): index = series.index
    dfs_out.append(pd.DataFrame(np.asarray(cols).T, index=index, columns=outputs))

df_in = pd.concat(dfs_in, sort=False)
df_out = pd.concat(dfs_out, sort=False)

print('{:6d} time series'.format(len(dfs_in)))
print('{:6d} total rows'.format(len(df_in)))

In [ ]:
std_in_cond, std_out_cond = StandardScaler(), StandardScaler()
net = MLPRegressor(hidden_layer_sizes=(64, 64, 32),
                   activation='relu',
                   solver='adam',
                   verbose=True,
                   early_stopping=True,
                   learning_rate_init=1e-4)
est_cond = Pipeline([('std', std_in_cond), ('net', net)])

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=FutureWarning)
    est_cond.fit(df_in, std_out_cond.fit_transform(df_out))

In [ ]:
# Save model
save = {
    'loss': est_cond['net'].loss_,
    'estimator': est_cond,
    'output_norm': std_out_cond,
    'actionvars': actionvars,
    'statevars': statevars,
    'inputs': inputs,
    'outputs': outputs,
    'lag': lag,
}
with open(path.join(bin_path, 'v2_condenser'), 'wb') as f:
    pickle.dump(save, f)

In [ ]:
# load model
with open(path.join(bin_path, 'v2_condenser'), 'rb') as f:
    save = pickle.load(f, fix_imports=False)
    est_cond = save['estimator']
    std_out_cond = save['output_norm']

In [ ]:
# Visualize model predictions
test_in, test_out = dfs_in[2], dfs_out[2]
pred = pd.DataFrame(std_out_cond.inverse_transform(est_cond.predict(test_in)),
                    index=test_out.index, columns=test_out.columns)

plt.subplot(4, 1, 1)
test_in.loc[:, ('TempCondIn')].plot(grid=True, style='g:', label='TempCondIn')
test_out.loc[:, ('TempCondIn')].plot(grid=True, style='-', label='TempCondIn-Next')
pred.loc[:, ('TempCondIn')].plot(grid=True, style='-', label='TempCondIn-Next-Pred')
plt.legend()
plt.subplot(4, 1, 2)
test_in.loc[:, ('TempCondOut')].plot(grid=True, style='g:', label='TempCondOut')
test_out.loc[:, ('TempCondOut')].plot(grid=True, style='-', label='TempCondOut-Next')
pred.loc[:, ('TempCondOut')].plot(grid=True, style='-', label='TempCondOut-Next-Pred')
plt.legend()
plt.subplot(4, 1, 3)
test_out.loc[:, ('PowChi')].plot(grid=True, label='PowChi')
pred.loc[:, ('PowChi')].plot(grid=True, label='PowChi-Pred')
plt.legend()
plt.subplot(4, 1, 4)
test_out.loc[:, ('PowFanA')].plot(grid=True, label='PowFanA')
pred.loc[:, ('PowFanA')].plot(grid=True, label='PowFanA-Pred')
plt.legend()

In [ ]:
point = df_in.loc['2020-07-10T1200-6'].values.reshape(1, -1)
x, y, z = model_surface(lambda x: std_out_cond.inverse_transform(est_cond.predict(x))[:,2],
                        X=point, vary_idx=(0, 3),vary_range=((65, 85), (75, 95)), vary_num=(20, 20))
ax = plot_surface(x,y,z, cmap=plt.cm.coolwarm)
ax.set_xlabel('TempCondInSetpoint')
ax.set_ylabel('TempAmbient')
ax.set_zlabel('TempCondIn-Next Cycle')

In [ ]:
print(df_in.loc['2020-07-10T1200-6'])